# RWKV Token Shift Experiment (Memory Finetune)
This continues off from `./TokenShift-D-basemodel.ipynb` to perform the full memory finetune & testing process

This is done generally in 3 Tune stages
- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.
- Tune 2: Low ctx size (512), Training with instruction & input masked. This forces the actual memory training on the output tokens.
- Tune 3: Mid ctx size (1024), stage 2, scaled up to 1024 context sizes.
- Tune 4: Mid ctx size (4096), stage 3, scaled up to 4096 context sizes.

In all cases, the input tokens is always masked. And we intentionally use the limited word set for memory training, which matches the same wordset used in the original memory evaluation of raven pretrained models. This is intentional to serve as both consistent comparision between experiments, and resonable training time.

One of the issue faced previously with an excessive large word set, is that the model would be required to see "new words" atleast a few time before being able to train the memory process. This drastically slowed down the process as the large word list meant the model was constantly spending time learning new words (instead of memory training).

If we want to increase the number / type of words the model can handle for memory training, that can be done later as a stage 4 memory tune if needed. But that exceeds the current requirements for the memory experiment process.

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Optional: Download the pretrained model
(if you want to skip the the basemodel train + instruct tune)


In [1]:
# # Init required dirs
# !mkdir -p ../../../../model/
# !mkdir -p ../../../../datapath/
# !mkdir -p ../../../../checkpoint/

# # Download the Stage2.pth file
# !rm -rf ../../../../model/TokenShift-D-Stage2.pth
# !cd ../../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/TokenShift-D-Stage2.pth
# !ls -alh ../../../../model/TokenShift-D-Stage2.pth

## Configure your environment settings
(!Important: you will need to rerun the below cell, if you restart your kernel)

In [2]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="TokenShift-D"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4wavenet/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/rwkv5x-tokenshift-exp-A/notebook/experiment/tokenshift-exp/TokenShift-D
TRAINER_DIR: /root/rwkv5x-tokenshift-exp-A/RWKV-v4wavenet
PROJECT_DIR: /root/rwkv5x-tokenshift-exp-A


## Tune 1 : Simple Memory instruct finetuning

- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.

In [3]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function. 
#
# Note that all document samples, are randomized between the target word count, 
# to half of the target word count.
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-5-count.jsonl  5  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-10-count.jsonl 10 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-15-count.jsonl 15 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-20-count.jsonl 20 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-25-count.jsonl 25 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-40-count.jsonl 40 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-50-count.jsonl 50 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-60-count.jsonl 80 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-80-count.jsonl 80 2500 &

# With a slight mix of the larger word count
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-100-count.jsonl 100 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-200-count.jsonl 200 2500 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 10 max words, 2500 samples - at ../dataset/word-10-count.jsonl
Generated JSONL file with - 15 max words, 2500 samples - at ../dataset/word-15-count.jsonl
Generated JSONL file with - 20 max words, 2500 samples - at ../dataset/word-20-count.jsonl
Generated JSONL file with - 25 max words, 2500 samples - at ../dataset/word-25-count.jsonl
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 5 max words, 5000 samples - at ../dataset/word-5-count.jsonl
Generated JSONL file with - 40 max words, 2500 samples - at ../dataset/word-40-count.jsonl
Generated JSONL file with - 50 max words, 2500 samples - at ../dataset/word-50-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-80-count.jsonl
Generated JSONL file with - 80 max words, 2500 samples - at ../dataset/word-60-count.jsonl
Generated JSONL file with - 100 max words, 2500 sample

In [4]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-1.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-D-mem-finetune-1/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 342.92it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5f88d0a1db77f214/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 205.45it/s]


In [5]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-1.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-1 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 126229482
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 126229482
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230725_003931-envps9kr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run TokenShift-D - Mem-Finetune-1 (bs=256, train-ctx=512, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-5X-Experiments
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-5X-Experiments/runs/envps9kr
Using /root/.cache/torch_extensions/py311_cu118 as PyTo

In [6]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/TokenShift-D-mem-finetune-1/last.ckpt" \
        "../model/TokenShift-D-Tune1.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/TokenShift-D-Tune1.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/TokenShift-D-mem-finetune-1/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/TokenShift-D-Tune1.pth
-rw-r--r-- 1 root root 5.3G Jul 25 01:22 ../model/TokenShift-D-Tune1.pth


In [7]:
# Lets do a memory eval
#
# Note that the expected performance "is not that great", as the model seems to be only loosely
# learning the memorization task, and the instruction propmt. And is seem to be acting more
# like an RNG based on the instruct. (Instead of the actual memorization task)
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-D-Tune1.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 85.0% similarity, with 17 matched token, and 3 token mismatch
## Model valid

## Tune 2 : Low ctx size (512), memory training

- Tune 2: Low ctx size (512), Training with instruction & input masked. This forces the actual memory training on the output tokens.

In [8]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We switch over to fully masked instruct+input, to properly learn the memorization task
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
for i in {5..95..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 5000 & 
done
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-100-count.jsonl 100 5000 &
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-200-count.jsonl 200 5000 &

#
# We mixin the shuffled word list, so that we ensure all words / tokens are learned
# however this might intrduce an exclusion bias (if seen this word, never repeat it), 
# so we limit the mixture of this data samples
#
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-10-count.jsonl 10 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-15-count.jsonl 15 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-25-count.jsonl 25 30 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-50-count.jsonl 50 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-75-count.jsonl 75 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-100-count.jsonl 100 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-200-count.jsonl 200 50 &

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 5000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 5 max words, 5000 samples - at ../dataset/gen-word-5-count.jsonl
Generated a single JSONL file with 3189 samples (30 token repeat) - 25 max words - at ../dataset/shuffle-word-25-count.jsonl
Generated a single JSONL file with 683 samples (50 token repeat) - 200 max words - at ../dataset/shuffle-word-200-count.jsonl
Generated a single JSONL file with 1328 samples (50 token repeat) - 100 max words - at ../dataset/shuffle-word-100-count.jsonl
Generated a single JSONL file with 5238 samples (20 token repeat) - 10 max words - at ../dataset/shuffle-word-10-count.jsonl
Generated JSONL file with - 10 max words, 5000 samples - at ../dataset/gen-word-10-count.jsonl
Generated a single JSONL file with 2619 samples (50 token repeat) - 50 max words - at ../dataset/shuffle-word-50-count.jsonl
Generated JSONL file with - 15 max words, 5000 samples 

In [9]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-2.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-D-mem-finetune-2/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 128.07it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a5ca6a42f9a28314/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 197.41it/s]


In [10]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-2.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-2 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 579741080
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 579741080
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230725_013840-6zdb6gci
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run TokenShift-D - Mem-Finetune-2 (bs=256, train-ctx=512, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-5X-Experiments
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-5X-Experiments/runs/6zdb6gci
Using /root/.cache/torch_extensions/py311_cu118 as PyTo

In [26]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/TokenShift-D-mem-finetune-2/last.ckpt" \
        "../model/TokenShift-D-Tune2.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/TokenShift-D-Tune2.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/TokenShift-D-mem-finetune-2/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/TokenShift-D-Tune2.pth
-rw-r--r-- 1 root root 5.3G Jul 25 04:36 ../model/TokenShift-D-Tune2.pth


In [27]:
# Lets do a memory eval 
#
# While not at its full potential, its memory ability should start emerging
#
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-D-Tune2.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model vali

## Tune 3 : Ramping up the ctx size (1024), memory training

- Tune 3: Mid ctx size (1024), same as tune 2, but extended in context size

This intentionally a much larger dataset, and lower learning rate to help ensure we push the model to its absolute limits.

In [56]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 1000 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 1000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 10 & 
done

#
# Ramping up the 50+ - 400 words dataset
# 
for i in {50..450..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 5 max words, 1000 samples - at ../dataset/gen-word-5-count.jsonl
Generated a single JSONL file with 648 samples (10 token repeat) - 40 max words - at ../dataset/shuffle-word-40-count.jsonl
Generated JSONL file with - 10 max words, 1000 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 15 max words, 1000 samples - at ../dataset/gen-word-15-count.jsonl
Generated JSONL file with - 20 max words, 1000 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 2 max words, 1000 samples - at ../dataset/word-2-count.jsonl
Generated JSONL file with - 35 max words, 1000 samples - at ../dataset/gen-word-35-count.jsonl
Generated a single JSONL file with 314 samples (20 token repeat) - 145 max words - at ../dataset/shuffle-word-145-count.jsonl
Generated a single JSONL file with 1307 samples (10 token repeat) - 20 max words - at ../dataset/shuffle-word-20-count.jsonl
Generated a single JSON

In [57]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-3.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-D-mem-finetune-3/"

Extracting data files: 100%|██████████████████████| 1/1 [00:00<00:00, 26.54it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d1a304ae1026187a/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 83.80it/s]


In [58]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-3.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-3 (bs=256, train-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=1024

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1289647921
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1289647921
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230725_054404-ur4a8zsu
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run TokenShift-D - Mem-Finetune-3 (bs=256, train-ctx=1024, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-5X-Experiments
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-5X-Experiments/runs/ur4a8zsu
Using /root/.cache/torch_extensions/py311_cu118 as P

In [59]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/TokenShift-D-mem-finetune-3/last.ckpt" \
        "../model/TokenShift-D-Tune3.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/TokenShift-D-Tune3.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/TokenShift-D-mem-finetune-3/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/TokenShift-D-Tune3.pth
-rw-r--r-- 1 root root 5.3G Jul 25 09:56 ../model/TokenShift-D-Tune3.pth


In [60]:
# Lets do a memory eval 
#
# We should start approaching the full potential of the model, unless its able to exceed 250 tokens of memory
#
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-D-Tune3.pth"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model vali

## Tune 4 : Ramping up the ctx size (4096), memory training

- Tune 4: Mid ctx size (4096), same as tune 4, but extended in context size

This intentionally a much larger dataset, and lower learning rate to help ensure we push the model to its absolute limits (supposingly)

In [61]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 1000 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 50+ - 2000 words dataset
# 
for i in {50..2000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated a single JSONL file with 175 samples (1 token repeat) - 15 max words - at ../dataset/shuffle-word-15-count.jsonl
Generated a single JSONL file with 56 samples (1 token repeat) - 45 max words - at ../dataset/shuffle-word-45-count.jsonl
Generated JSONL file with - 10 max words, 100 samples - at ../dataset/gen-word-10-count.jsonl
Generated JSONL file with - 20 max words, 100 samples - at ../dataset/gen-word-20-count.jsonl
Generated JSONL file with - 5 max words, 100 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 30 max words, 100 samples - at ../dataset/gen-word-30-count.jsonl
Generated JSONL file with - 15 max words, 100 samples - at ../dataset/gen-word-15-count.jsonl
Generated a single JSONL file with 130 samples (1 token repeat) - 20 max words - at ../dataset/shuffle-word-20-count.jsonl
Generated JSONL file with - 25 max words, 100 samples - at ../dataset/gen-word-25-count.jsonl
Generated JSONL file with - 2 

In [62]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-4.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-D-mem-finetune-4/"

Extracting data files: 100%|██████████████████████| 1/1 [00:00<00:00,  5.86it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-862270205dc70075/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 36.60it/s]


In [63]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-4.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-4 (bs=256, train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=4096

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 2929658149
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 2929658149
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230725_101602-mlvshwzg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run TokenShift-D - Mem-Finetune-4 (bs=256, train-ctx=4096, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-5X-Experiments
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-5X-Experiments/runs/mlvshwzg
Using /root/.cache/torch_extensions/py311_cu118 as P

In [64]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/TokenShift-D-mem-finetune-4/last.ckpt" \
        "../model/TokenShift-D-Tune4.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/TokenShift-D-Tune4.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/TokenShift-D-mem-finetune-4/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/TokenShift-D-Tune4.pth
-rw-r--r-- 1 root root 5.3G Jul 26 13:54 ../model/TokenShift-D-Tune4.pth


In [65]:
# Lets do a memory eval 
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-D-Tune4.pth" 

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model vali

## Tune 5 : Ramping up the ctx size (8192), memory training

- Tune 5: Mid ctx size (8192), same as tune 4, but extended in context size again!

This intentionally a much larger dataset, with increased leaarning rate, specifically for L24+ tokenshift models, who have been shown to clear Tune 4

In [70]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 1000 &
for i in {5..95..10} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 100+ - 8200 words dataset
# 
for i in {100..8200..100} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 1000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 10 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 95 max words, 100 samples - at ../dataset/gen-word-95-count.jsonl
Generated JSONL file with - 25 max words, 100 samples - at ../dataset/gen-word-25-count.jsonl
Generated a single JSONL file with 49 samples (1 token repeat) - 55 max words - at ../dataset/shuffle-word-55-count.jsonl
Generated a single JSONL file with 31 samples (1 token repeat) - 85 max words - at ../dataset/shuffle-word-85-count.jsonl
Generated a single JSONL file with 78 samples (1 token repeat) - 35 max words - at ../dataset/shuffle-word-35-count.jsonl
Generated JSONL file with - 5 max words, 100 samples - at ../dataset/gen-word-5-count.jsonl
Generated JSONL file with - 85 max words, 100 samples - at ../dataset/gen-word-85-count.jsonl
Generated JSONL file with - 65 max words, 100 samples - at ../dataset/gen-word-65-count.jsonl
Generated a single JSONL file with 558 samples (1 token repeat) - 5 max words - at ../dataset/shuffle-word-5-count.jsonl
Gener

In [71]:
# Lets pre tokenize the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-5.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/TokenShift-D-mem-finetune-5/"

Extracting data files: 100%|██████████████████████| 1/1 [00:00<00:00, 25.82it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-c7c94d4eb4ce27c1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 62.66it/s]


In [72]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-D-mem-finetune-5.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-5 (bs=256, train-ctx=2*4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=4096

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1558535245
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1558535245
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230726_150601-kvg8wm8c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run TokenShift-D - Mem-Finetune-5 (bs=256, train-ctx=2*4096, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-5X-Experiments
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-5X-Experiments/runs/kvg8wm8c
Using /root/.cache/torch_extensions/py311_cu118 as

In [74]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/TokenShift-D-mem-finetune-5/last.ckpt" \
        "../model/TokenShift-D-Tune5.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/TokenShift-D-Tune5.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/TokenShift-D-mem-finetune-5/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/TokenShift-D-Tune5.pth
-rw-r--r-- 1 root root 5.3G Jul 27 00:53 ../model/TokenShift-D-Tune5.pth


In [75]:
# Lets do a memory eval 
!python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-D-Tune5.pth" 

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
###
### Model validation start ###
###
## Model validation for 5 tokens : 100.0% similarity, with 5 matched token, and 0 token mismatch
## Model validation for 10 tokens : 100.0% similarity, with 10 matched token, and 0 token mismatch
## Model validation for 15 tokens : 100.0% similarity, with 15 matched token, and 0 token mismatch
## Model validation for 20 tokens : 100.0% similarity, with 20 matched token, and 0 token mismatch
## Model vali